In [1]:
import pandas as pd

## Loading Data

In [2]:
training_data = pd.read_csv("train.csv")
training_data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [3]:
training_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [4]:
# Adding a new column to mark a cabin was listed or not for the passenger
training_data['Cabin_Present'] = training_data['Cabin'].notna().astype(int)

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [6]:
training_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   PassengerId    891 non-null    int64  
 1   Survived       891 non-null    int64  
 2   Pclass         891 non-null    int64  
 3   Name           891 non-null    object 
 4   Sex            891 non-null    object 
 5   Age            714 non-null    float64
 6   SibSp          891 non-null    int64  
 7   Parch          891 non-null    int64  
 8   Ticket         891 non-null    object 
 9   Fare           891 non-null    float64
 10  Cabin          204 non-null    object 
 11  Embarked       889 non-null    object 
 12  Cabin_Present  891 non-null    int64  
dtypes: float64(2), int64(6), object(5)
memory usage: 90.6+ KB


## Filling missing data and training a logistic regression model
The *Age* and *Embarked* columns have some missing data. The missing age values will be filled by the average age and the missing embarked location will be filled by the most frequent location.

In [7]:
# Separate features and target
X = training_data.drop(["PassengerId", "Survived", "Name", "Ticket", "Cabin"], axis=1)
y = training_data["Survived"]

# Define preprocessing for numerical features (impute missing values, then scale)
numerical_features = ["Age", "SibSp", "Parch", "Fare"]
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# Define preprocessing for categorical features (impute missing values, then one-hot encode)
categorical_features = ["Pclass", "Sex", "Embarked"]
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder="passthrough" 
)

# Create and train the model pipeline
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(max_iter=200))
])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)

Accuracy: 0.8044692737430168


## Predicting the output

In [8]:
predicting_data = pd.read_csv("test.csv")
predicting_data

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [9]:
predicting_data["Cabin_Present"] = predicting_data['Cabin'].notna().astype(int)

In [10]:
# Drop columns that are not needed
X_new = predicting_data.drop(["PassengerId", "Name", "Ticket", "Cabin"], axis=1)

# Make predictions on the new data using the trained model
y_new_pred = model.predict(X_new)

# Optionally, save the predictions to a file
predicting_data['Predicted_Survived'] = y_new_pred
predicting_data[["PassengerId", "Predicted_Survived"]].to_csv('predictions.csv', index=False)

# Display the predictions
print(predicting_data[['PassengerId', 'Predicted_Survived']])

     PassengerId  Predicted_Survived
0            892                   0
1            893                   0
2            894                   0
3            895                   0
4            896                   1
..           ...                 ...
413         1305                   0
414         1306                   1
415         1307                   0
416         1308                   0
417         1309                   0

[418 rows x 2 columns]


## Kaggle submission result
The predictions CSV file that was genereated was uploaded to Kaggle to check the result. A score of 0.75 was obtained.

I'll be using GridSearchCV in the next step to improve the score.

## Improving the model accuracy using GridSearchCV

In [11]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'classifier__C': [0.1, 1, 10, 100],  # Hyperparameters for Logistic Regression
    'classifier__solver': ['liblinear', 'saga']
}

# Initialize Grid Search
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy')

# Fit Grid Search
grid_search.fit(X_train, y_train)

# Best parameters and best score
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Score:", grid_search.best_score_)

Best Parameters: {'classifier__C': 1, 'classifier__solver': 'liblinear'}
Best Cross-Validation Score: 0.8061164187924753


## Making predictions using the best parameters

In [12]:
# Load the new data
predicting_data2 = pd.read_csv('test.csv')

# Encode the Cabin column as a binary indicator
predicting_data2['Cabin_Present'] = predicting_data2['Cabin'].notna().astype(int)

# Drop columns that are not needed
X_new2 = predicting_data2.drop(["PassengerId", "Name", "Ticket", "Cabin"], axis=1)

# Apply the best parameters found by Grid Search
best_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(C=1, solver='liblinear', max_iter=200))
])

# Fit the model on the entire training data
best_model.fit(X, y)

# Make predictions on the new data
y_new_pred2 = best_model.predict(X_new2)

# Optionally, save the predictions to a file
predicting_data2['Survived'] = y_new_pred2
predicting_data2[["PassengerId", "Survived"]].to_csv('predictions2.csv', index=False)

# Display the predictions
print(predicting_data2[['PassengerId', 'Survived']])

     PassengerId  Survived
0            892         0
1            893         0
2            894         0
3            895         0
4            896         1
..           ...       ...
413         1305         0
414         1306         1
415         1307         0
416         1308         0
417         1309         0

[418 rows x 2 columns]


## Kaggle re-submission result
The score with the improved model was slightly higher than the previous score - 0.756.

For future improvements to the model, following can be considered:
* feature engineering
* selecting a different methodology than Logistic regression